In [10]:
import os
import pandas as pd

REMOVE_COLUMNS_REGION = ["CODIGO (WMO):", "DATA DE FUNDACAO:", "ESTACAO:"]
GET_COLUMNS_CLIMATE = [
    "Data",
    "Hora UTC",
    "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)",
    "VENTO, VELOCIDADE HORARIA (m/s)",
    "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",
    "RADIACAO GLOBAL (Kj/m²)",
    "UMIDADE RELATIVA DO AR, HORARIA (%)",
]


#### Leitura dos arquivos sobre o clima

Arquivos baixados do site [inmet](https://portal.inmet.gov.br/dadoshistoricos) (arquivos de 2022)

In [11]:
climate_dir = "./data/2022"
list_df_station = []
list_df_climate = []

count = 0

for file_name in os.listdir(climate_dir):
    # Nossa tabela fato, utiliza o nome da UF, latitude, longitude e altitude
    df_station = pd.read_csv(
        f"{climate_dir}/{file_name}",
        delimiter=";",
        encoding="Windows 1252",
        nrows=7,
    ).set_index("REGIAO:").transpose().reset_index(drop=True)

    df_station = df_station.drop(columns=REMOVE_COLUMNS_REGION)

    df_station = df_station.rename(columns={"UF:":"UF",
                                            "LATITUDE:":"latitude",
                                            "LONGITUDE:":"longitude",
                                            "ALTITUDE:":"altitude"})


    df_climate = pd.read_csv(
        f"{climate_dir}/{file_name}",
        delimiter=";",
        encoding="Windows 1252",
        skiprows=8,
    ).dropna(how="all", subset=GET_COLUMNS_CLIMATE[2:]).get(GET_COLUMNS_CLIMATE)

    df_climate = df_climate.rename_axis("id")

    df_climate = df_climate.rename(columns={"CODIGO (WMO)": "id",
                                  "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)": "precipTotalHorario",
                                  "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)": "tempAr",
                                  "VENTO, VELOCIDADE HORARIA (m/s)": "ventoVeloHoraria",
                                  "RADIACAO GLOBAL (Kj/m²)": "radGlobal",
                                  "UMIDADE RELATIVA DO AR, HORARIA (%)": "umidRelAr"})
    
    #Adicionando id da FK
    df_climate["id_regiao"] = count
    count = count+1
    


    list_df_station.append(df_station)
    list_df_climate.append(df_climate)

In [12]:
df_all_station = pd.concat(list_df_station, ignore_index=True)
df_all_station = df_all_station.rename_axis("id")
df_all_station.to_csv("./data/dim_regiao.csv", sep=";")

In [13]:
df_all_climate = pd.concat(list_df_climate, ignore_index=True)
df_all_climate = df_all_climate.rename_axis("id")
df_all_climate.to_csv("./data/fato_dados_meteorologicos.csv", sep=";")